## Finlex API

In [40]:
# https://data.finlex.fi/fi/rest-api

# Legislation
# https://data.finlex.fi/eli/sd/2004/1.html
# https://data.finlex.fi/eli/sd/2004/1.jsonld

# Case law
# https://data.finlex.fi/ecli/kho/2011/1.html
# https://data.finlex.fi/ecli/kho/2011/1.jsonld

In [37]:
import os
import requests
import json
import re
import string
import pandas as pd

In [38]:
def clean(s):
    s = re.sub("\n", "", s)
    s = s.translate(str.maketrans('', '', string.punctuation + string.digits))
    s = re.sub("§(\S+)?", "", s)
    s = re.sub(" +", " ", s)
    s = re.sub("(^| ).( |$)", " ", s)
    s = re.sub("–", "", s)
    s = s.strip()
    s = s.lower()
    return(s)

In [46]:
data_type = "ecli/kko"

for year in range(1700, 2020):
    
    data = []
    
    url = "https://data.finlex.fi/" + data_type + "/" + str(year) + ".jsonld"
    r = requests.get(url)
    
    d_json = json.loads(r.text)
    
    if "error" in d_json.keys():
        continue
        
    for i in range(0, len(d_json['@graph'])):
        
        if data_type == "eli/sd":
            d = d_json['@graph'][i]['temporalVersions'][0]['languageVersion'][0]
            text = clean(d['title_fi'][0] + " " + d['content'])
        elif data_type == "ecli/kho":
            d = d_json['@graph'][i]['languageVersion'][0]
            text = clean(d['abstract_fi'][0])
        else:
            d = d_json['@graph'][i]['languageVersion'][0]
            
            if 'hasFormat' in d.keys():
                text = clean(d['abstract_fi'][0] + " " + d['hasFormat'][0]['content_fi'])
            elif 'content_fi' in d.keys():
                text = clean(d['abstract_fi'][0] + " " + d['content_fi'])
            else:
                text = clean(d['abstract_fi'][0])

        data.append(
            {
                "id" : d['@id'],
                "type" : d['@type'],
                "year" : year,
                "text" : text,
                "url" : url
            }
        )
    
    if data:
        print(str(year) + ": " + str(len(data)))
        with open('data/' + data_type.split("/")[1] + '/' + str(year) + '.json', 'w') as f:
            json.dump(data, f)

1980: 144
1981: 187
1982: 200
1983: 197
1984: 234
1985: 200
1986: 171
1987: 136
1988: 140
1989: 157
1990: 180
1991: 189
1992: 197
1993: 170
1994: 145
1995: 215
1996: 156
1997: 211
1998: 164
1999: 129
2000: 129
2001: 138
2002: 124
2003: 138
2004: 136
2005: 145
2006: 110
2007: 103
2008: 119
2009: 94
2010: 96
2011: 111
2012: 109
2013: 102
2014: 104
2015: 105
2016: 100
2017: 98
2018: 92
2019: 89


In [11]:
with open("data/sd/" + os.listdir("data/sd")[100], 'r') as f:
    d = json.load(f)